In [1]:
import re
import json
import os

In [2]:
LIST_OF_DOCS = ["../raw_quizzes/GPT-made_quizzes/filtered/"+f for f in os.listdir("../raw_quizzes/GPT-made_quizzes/filtered") if f.endswith('.txt')]

print(LIST_OF_DOCS)
print(len(LIST_OF_DOCS))

['../raw_quizzes/GPT-made_quizzes/filtered/analysis-itembank-sim-2-Ori-laryngeal_contrasts-filtered.txt', '../raw_quizzes/GPT-made_quizzes/filtered/analysis-itembank-sim-3-Sziget-vowels-filtered.txt', '../raw_quizzes/GPT-made_quizzes/filtered/analysis-itembank-sim-4-Sziget-vowels_and_stress-filtered.txt', '../raw_quizzes/GPT-made_quizzes/filtered/analysis-itembank-sim-5-Kiss-phrasal_stress_and_intonation-filtered.txt', '../raw_quizzes/GPT-made_quizzes/filtered/analysis-itembank-sim-9-Attila-variation-filtered.txt']
5


In [3]:
for doc in LIST_OF_DOCS:
    with open(doc, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()

    # Split the content into blocks using '== cat' as a separator
    blocks = re.split(r'\n== cat ', content)
    data = []

    # Process each block
    for block in blocks:        
        lines = block.split('\n')

        question_lines = []
        answer_lines_raw = [] # still with hashtags
        answer_lines = [] # with letters
        option_labels = ['a)', 'b)', 'c)', 'd)']

        category = lines[0].strip().replace("== cat ", "") # deal with categories, replacement necessary for q1
        question = ""
        correct_answer = ""

        for line in lines:
            if re.search(r"\[\[.+\]\]", line): # the open questions
                question = line
                correct_answers = re.findall(r"\[?\[\[(.*?)\]\]\]?", line)
                question = re.sub(r"\[?\[\[.*?\]\]\]?$", "", line.strip()) # if the [[]] are at the end, then they turn into nothing
                question = re.sub(r"\[?\[\[.*?\]\]\]?", "...", question) # somewhere in the middle they turn into three dots
                question = re.sub(r"\.\.\.\.", "...", question.strip()) # 4 dots turn into three
                correct_answer = "|||".join(correct_answers)
            elif re.match(r"^[A-Z]", line): # if starts with a capital letter, then it's a question
                question = line.strip()
            elif re.match(r"^#", line): # if starts with a # then it's an answer
                answer_lines_raw.append(line.strip())
        
        for idx, answer_line in enumerate(answer_lines_raw):
            # Check if the answer is marked as correct
            if answer_line.startswith('# !'):
                is_correct = True
                answer_text = answer_line[3:].strip() # Remove the '# !' prefix
            else:
                is_correct = False
                answer_text = answer_line[2:].strip() # Remove the '#' prefix
            
            # Assign a letter to the option
            option_label = option_labels[idx]
            option_entry = f"{option_label} {answer_text}"
            answer_lines.append(option_entry)
            
            # Collect the correct answer(s)
            if is_correct:
                correct_answer = option_entry
                answer_text = ""



        item = {
        'question': question,
        'category': category,
        'answers': answer_lines,
        'correct answer(s)': correct_answer
        }

        data.append(item)


    # Output the data to a JSON file
    with open(f"itembank-{itembank_number}.json", 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, ensure_ascii=False, indent=2)